In [1]:
import re
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline
import os, sys
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA 
import plotly.express as px
from sklearn.metrics import plot_confusion_matrix

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score,KFold,StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,average_precision_score,recall_score,roc_auc_score
from sklearn.preprocessing import RobustScaler,StandardScaler,LabelEncoder,MinMaxScaler


In [2]:
train = pd.read_csv("/content/covid19_tweets.csv")
train

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125096,Channel 1 Atlanta News NOW,"Atlanta, GA","Apart of #Atlanta, #America, & the #World for ...",2008-12-28 22:17:21,3574,4581,15684,False,2020-08-13 06:40:54,WATCH LIVE #Breaking #news #severe #wx #covid...,"['Breaking', 'news', 'severe', 'wx', 'covid19'...",Periscope,False
125097,Agility Africa,Across Africa,Agility funds and develops world-class warehou...,2015-05-18 13:10:39,673,574,533,False,2020-08-13 06:40:52,#Agility donates warehouse space to fight #COV...,"['Agility', 'COVID19']",Twitter Web App,False
125098,Hartmut Herrmann,"Leipzig, Germany","Atmospheric Chemistry Prof. & Dept Head , Phys...",2018-02-23 18:46:26,585,333,4235,False,2020-08-13 06:40:51,"Follow-up fm yesterday, HT @johnwenger9. Reall...",NaN,Twitter Web App,False
125099,Shortpedia,"New Delhi, India",Fresh handpicked content everyday.\n\n#EvolveE...,2017-06-26 11:27:15,739,166,60,False,2020-08-13 06:40:47,#Russia has rejected the reports raising safet...,"['Russia', 'coronavirusvaccine']",Twitter Web App,False


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125101 entries, 0 to 125100
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_name         125101 non-null  object
 1   user_location     99909 non-null   object
 2   user_description  118144 non-null  object
 3   user_created      125101 non-null  object
 4   user_followers    125101 non-null  int64 
 5   user_friends      125101 non-null  int64 
 6   user_favourites   125101 non-null  int64 
 7   user_verified     125101 non-null  bool  
 8   date              125101 non-null  object
 9   text              125101 non-null  object
 10  hashtags          89130 non-null   object
 11  source            125065 non-null  object
 12  is_retweet        125101 non-null  bool  
dtypes: bool(2), int64(3), object(8)
memory usage: 10.7+ MB


In [4]:
train.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [5]:
train.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [6]:
train.drop('user_location', axis=1, inplace=True)
train.drop('user_created', axis=1, inplace=True)
train.drop('user_followers', axis=1, inplace=True)
train.drop('user_friends', axis=1, inplace=True)
train.drop('user_favourites', axis=1, inplace=True)
train.drop('source', axis=1, inplace=True)
train.drop('is_retweet', axis=1, inplace=True)


In [ ]:
train.head(100)

,user_name,user_description,user_verified,date,text,hashtags
0,ᏉᎥ☻լꂅϮ,wednesday addams as a disney princess keepin i...,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN
1,Tom Basile 🇺🇸,"Husband, Father, Columnist & Commentator. Auth...",True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN
2,Time4fisticuffs,#Christian #Catholic #Conservative #Reagan #Re...,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19']
3,ethel mertz,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19']
4,DIPR-J&K,🖊️Official Twitter handle of Department of Inf...,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']"
...,...,...,...,...,...,...
95,Safiy,"Detroit life. Creative Beast, Animation, Visua...",False,2020-07-25 12:24:20,Shout out to Earth. Hands down the worst react...,NaN
96,TNCoronaUpdates,Data Analyzer | Stay Connected For All Offici...,False,2020-07-25 12:24:19,#TamilNadu | #COVID19 | 25th July\n\n● TN - ...,"['TamilNadu', 'COVID19']"
97,CraigBrigham MebaneNC #864511320 #GoodTrouble,"Fmr Army Security Agency, Meritorious Service....",False,2020-07-25 12:24:19,@WLOS_13 @wcti12 @thecw22 @myrdc @ABC45TV Plea...,NaN
98,The GAA,Gaelic Athletic Association/Cumann Lúthchleas ...,True,2020-07-25 12:24:17,You can all #PlayYourPart in the fight against...,"['PlayYourPart', 'Covid19', 'InThisTogether']"


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125101 entries, 0 to 125100
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_name         125101 non-null  object
 1   user_description  118144 non-null  object
 2   user_verified     125101 non-null  bool  
 3   date              125101 non-null  object
 4   text              125101 non-null  object
 5   hashtags          89130 non-null   object
dtypes: bool(1), object(5)
memory usage: 4.9+ MB


In [8]:
train.drop('user_description', axis=1, inplace=True)
train.drop('hashtags', axis=1, inplace=True)

In [9]:
train.head(500)

,user_name,user_verified,date,text
0,ᏉᎥ☻լꂅϮ,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...
1,Tom Basile 🇺🇸,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2,Time4fisticuffs,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...
3,ethel mertz,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...
4,DIPR-J&K,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...
...,...,...,...,...
495,Coburg Consulting Group Ltd,False,2020-07-25 12:11:10,As Somalia heads to political campaign and lat...
496,Judy Cheesey,False,2020-07-25 12:11:10,@jjauthor Can someone give this twirp and his ...
497,Mauricio Ramirez Villegas,False,2020-07-25 12:11:08,The @UN is helping families use the opportunit...
498,beingData,False,2020-07-25 12:11:08,Spike in U.S. Cases Far Outpaces Testing Expan...


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125101 entries, 0 to 125100
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_name      125101 non-null  object
 1   user_verified  125101 non-null  bool  
 2   date           125101 non-null  object
 3   text           125101 non-null  object
dtypes: bool(1), object(3)
memory usage: 3.0+ MB


In [11]:
def remove_pattern(text,pattern):
    
    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)
    
    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)
    
    return text

In [12]:
train['New Text'] = np.vectorize(remove_pattern)(train['text'], "@[\w]*")

train.head()

,user_name,user_verified,date,text,New Text
0,ᏉᎥ☻լꂅϮ,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,If I smelled the scent of hand sanitizers toda...
1,Tom Basile 🇺🇸,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,Hey PR and - wouldn't it have made more sens...
2,Time4fisticuffs,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,Trump never once claimed #COVID19 was a hoa...
3,ethel mertz,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,The one gift #COVID19 has give me is an appre...
4,DIPR-J&K,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,25 July : Media Bulletin on Novel #CoronaVirus...


In [13]:
train['New Text up'] = np.vectorize(remove_pattern)(train['New Text'], "#[\w]*")

In [14]:
train.head()

,user_name,user_verified,date,text,New Text,New Text up
0,ᏉᎥ☻լꂅϮ,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,If I smelled the scent of hand sanitizers toda...,If I smelled the scent of hand sanitizers toda...
1,Tom Basile 🇺🇸,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,Hey PR and - wouldn't it have made more sens...,Hey PR and - wouldn't it have made more sens...
2,Time4fisticuffs,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,Trump never once claimed #COVID19 was a hoa...,Trump never once claimed was a hoax. We al...
3,ethel mertz,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,The one gift #COVID19 has give me is an appre...,The one gift has give me is an appreciation ...
4,DIPR-J&K,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,25 July : Media Bulletin on Novel #CoronaVirus...,25 July : Media Bulletin on Novel \n … htt...


In [15]:
train.drop('New Text', axis=1, inplace=True)
train.head(100)

,user_name,user_verified,date,text,New Text up
0,ᏉᎥ☻լꂅϮ,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,If I smelled the scent of hand sanitizers toda...
1,Tom Basile 🇺🇸,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,Hey PR and - wouldn't it have made more sens...
2,Time4fisticuffs,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,Trump never once claimed was a hoax. We al...
3,ethel mertz,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,The one gift has give me is an appreciation ...
4,DIPR-J&K,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,25 July : Media Bulletin on Novel \n … htt...
...,...,...,...,...,...
95,Safiy,False,2020-07-25 12:24:20,Shout out to Earth. Hands down the worst react...,Shout out to Earth. Hands down the worst react...
96,TNCoronaUpdates,False,2020-07-25 12:24:19,#TamilNadu | #COVID19 | 25th July\n\n● TN - ...,"| | 25th July\n\n● TN - 6,988 (Highest Spi..."
97,CraigBrigham MebaneNC #864511320 #GoodTrouble,False,2020-07-25 12:24:19,@WLOS_13 @wcti12 @thecw22 @myrdc @ABC45TV Plea...,Please tell us you not going to run that ...
98,The GAA,True,2020-07-25 12:24:17,You can all #PlayYourPart in the fight against...,You can all in the fight against . We’re all...


In [16]:
train['New Text up'] = train['New Text up'].str.replace("[^a-zA-Z#]", " ")

In [17]:
train.head(100)

,user_name,user_verified,date,text,New Text up
0,ᏉᎥ☻լꂅϮ,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,If I smelled the scent of hand sanitizers toda...
1,Tom Basile 🇺🇸,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,Hey PR and wouldn t it have made more sens...
2,Time4fisticuffs,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,Trump never once claimed was a hoax We al...
3,ethel mertz,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,The one gift has give me is an appreciation ...
4,DIPR-J&K,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,July Media Bulletin on Novel http...
...,...,...,...,...,...
95,Safiy,False,2020-07-25 12:24:20,Shout out to Earth. Hands down the worst react...,Shout out to Earth Hands down the worst react...
96,TNCoronaUpdates,False,2020-07-25 12:24:19,#TamilNadu | #COVID19 | 25th July\n\n● TN - ...,th July TN Highest Spike...
97,CraigBrigham MebaneNC #864511320 #GoodTrouble,False,2020-07-25 12:24:19,@WLOS_13 @wcti12 @thecw22 @myrdc @ABC45TV Plea...,Please tell us you not going to run that ...
98,The GAA,True,2020-07-25 12:24:17,You can all #PlayYourPart in the fight against...,You can all in the fight against We re all...


In [18]:
train['New Text up'] = train['New Text up'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [19]:
train.head(10)

,user_name,user_verified,date,text,New Text up
0,ᏉᎥ☻լꂅϮ,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,smelled scent hand sanitizers today someone pa...
1,Tom Basile 🇺🇸,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,wouldn have made more sense have players their...
2,Time4fisticuffs,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,Trump never once claimed hoax claim that this ...
3,ethel mertz,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,gift give appreciation simple things that were...
4,DIPR-J&K,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,July Media Bulletin Novel https EEcsJHh
5,🎹 Franz Schubert,False,2020-07-25 12:27:06,#coronavirus #covid19 deaths continue to rise....,deaths continue rise almost ever Politicians b...
6,hr bartender,False,2020-07-25 12:27:03,How #COVID19 Will Change Work in General (and ...,Will Change Work General recruiting specifical...
7,Derbyshire LPC,False,2020-07-25 12:27:00,You now have to wear face coverings when out s...,have wear face coverings when shopping this in...
8,Prathamesh Bendre,False,2020-07-25 12:26:59,Praying for good health and recovery of @Chouh...,Praying good health recovery
9,Member of Christ 🇨🇳🇺🇸🇮🇳🇮🇩🇧🇷🇳🇬🇧🇩🇷🇺,False,2020-07-25 12:26:54,POPE AS GOD - Prophet Sadhu Sundar Selvaraj. W...,POPE Prophet Sadhu Sundar Selvaraj Watch here ...


In [20]:
tokenized_tweet = train['New Text up'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [smelled, scent, hand, sanitizers, today, some...
1    [wouldn, have, made, more, sense, have, player...
2    [Trump, never, once, claimed, hoax, claim, tha...
3    [gift, give, appreciation, simple, things, tha...
4       [July, Media, Bulletin, Novel, https, EEcsJHh]
Name: New Text up, dtype: object

In [21]:

from nltk import PorterStemmer

ps = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [ps.stem(i) for i in x])

tokenized_tweet.head()

0    [smell, scent, hand, sanit, today, someon, pas...
1    [wouldn, have, made, more, sens, have, player,...
2    [trump, never, onc, claim, hoax, claim, that, ...
3    [gift, give, appreci, simpl, thing, that, were...
4        [juli, media, bulletin, novel, http, eecsjhh]
Name: New Text up, dtype: object

In [22]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

train['New Text up'] = tokenized_tweet
train.head()

,user_name,user_verified,date,text,New Text up
0,ᏉᎥ☻լꂅϮ,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,smell scent hand sanit today someon past would...
1,Tom Basile 🇺🇸,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,wouldn have made more sens have player their r...
2,Time4fisticuffs,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,trump never onc claim hoax claim that thi effo...
3,ethel mertz,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,gift give appreci simpl thing that were alway ...
4,DIPR-J&K,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,juli media bulletin novel http eecsjhh


In [23]:
train.drop('text', axis=1, inplace=True)
train.head(10)

,user_name,user_verified,date,New Text up
0,ᏉᎥ☻լꂅϮ,False,2020-07-25 12:27:21,smell scent hand sanit today someon past would...
1,Tom Basile 🇺🇸,True,2020-07-25 12:27:17,wouldn have made more sens have player their r...
2,Time4fisticuffs,False,2020-07-25 12:27:14,trump never onc claim hoax claim that thi effo...
3,ethel mertz,False,2020-07-25 12:27:10,gift give appreci simpl thing that were alway ...
4,DIPR-J&K,False,2020-07-25 12:27:08,juli media bulletin novel http eecsjhh
5,🎹 Franz Schubert,False,2020-07-25 12:27:06,death continu rise almost ever politician busi...
6,hr bartender,False,2020-07-25 12:27:03,will chang work gener recruit specif http bjzx...
7,Derbyshire LPC,False,2020-07-25 12:27:00,have wear face cover when shop thi includ visi...
8,Prathamesh Bendre,False,2020-07-25 12:26:59,pray good health recoveri
9,Member of Christ 🇨🇳🇺🇸🇮🇳🇮🇩🇧🇷🇳🇬🇧🇩🇷🇺,False,2020-07-25 12:26:54,pope prophet sadhu sundar selvaraj watch here ...


In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125101 entries, 0 to 125100
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_name      125101 non-null  object
 1   user_verified  125101 non-null  bool  
 2   date           125101 non-null  object
 3   New Text up    125101 non-null  object
dtypes: bool(1), object(3)
memory usage: 3.0+ MB
